In [6]:
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
import numpy as np

#Settings
NumberOfIterations = 4
NumberOfSegments = 10
PowerIncrement = -0.5
CoefficientIncrement = -0.5

#Don't touch
LowerBound = 0
UpperBound = 1

class Integration:
    def __init__(self,f,a,b,N):
        self.f = f
        self.a = a
        self.b = b
        self.N = N

    def Trapz(self):    
        SubIntervals = self.GetSubInvtervals()
        #y = self.f(x)
        RightEndPoints = SubIntervals[1:]
        LeftEndPoints = SubIntervals[:-1]
        TotalArea = 0
        SegmentWidth = (self.b - self.a)/self.N
        for i in range(len(RightEndPoints)):
            TopperTriangleArea = SegmentWidth/2 * (self.f(RightEndPoints[i]) - self.f(LeftEndPoints[i]))
            RectangleArea = 0
            if RightEndPoints[i] > LeftEndPoints[i]:
                RectangleArea = SegmentWidth * self.f(LeftEndPoints[i])
            else:
                RectangleArea = SegmentWidth * self.f(RightEndPoints[i])
            TotalArea = TotalArea + RectangleArea + TopperTriangleArea
        return TotalArea

    def GetSubInvtervals(self):
        SubIntervalList = []
        for i in range(0,self.N+1):
            Value = self.a + i*(self.b - self.a)/self.N
            SubIntervalList.append(Value)    
        return SubIntervalList

    def Simps(self):
        SegmentWidth = (self.b - self.a)/self.N
        Sum = self.f(self.b) + self.f(self.a)
        for i in range(1,int(self.N)):
            if i % 2:
                Sum += 4*self.f(self.a+i*SegmentWidth)
            else:
                Sum += 2*self.f(self.a+i*SegmentWidth)
        return Sum*SegmentWidth/3

class FunctionHandling:
    def __init__(self, Coefficient, Power):
        self.Coefficient = Coefficient
        self.Power = Power
   
    def GetFunction(self):
        return lambda x : self.Coefficient*x**self.Power
    def GetValue(self, LowerBound, UpperBound):
        LowerIntegral = None
        if LowerBound == 0:
            LowerIntegral = 0
        else:
            LowerIntegral = ((self.Coefficient)*LowerBound**(self.Power+1))/(self.Power+1)
        
        UpperIntegral = None
        if UpperBound == 0:
            UpperIntegral = 0
        else:
            UpperIntegral = ((self.Coefficient)*UpperBound**(self.Power+1))/(self.Power+1)
        return (UpperIntegral - LowerIntegral)

class Graph:
    def __init__(self, XData, YData, ZData):
        self.XData = XData
        self.YData = YData
        self.ZData = ZData
    def ThreeDimensionalGraph(self, Name):
        plt.tight_layout()
        fig = plt.figure()
        ax = plt.axes(projection='3d')  
        ax.set_xlabel('Coefficient')
        ax.set_ylabel('Power')
        ax.set_zlabel('Error (%)')
        ax.set_title(Name)
        ax.plot_wireframe(self.XData, self.YData, self.ZData, color='blue')
        ax.dist = 12

class Main:
    def __init__(self, LowerBound, UpperBound, NumberOfSegments, NumberOfIterations):
        self.LowerBound = LowerBound
        self.UpperBound = UpperBound
        self.NumberOfSegments = NumberOfSegments
        self.NumberOfIterations = NumberOfIterations
        self.XData = [] #CoefficientList
        for i in range(NumberOfIterations):
            ListToAppend = []
            for j in range(NumberOfIterations):
                ListToAppend.append((i+1)*CoefficientIncrement)
            ListToAppend = np.array(ListToAppend)
            self.XData.append(ListToAppend)
        self.YData = []#Powerlist
        for i in range(NumberOfIterations):
            ListToAppend = []
            for j in range(NumberOfIterations):
                ListToAppend.append(j*PowerIncrement)
            ListToAppend = np.array(ListToAppend)
            self.YData.append(ListToAppend)
        self.ZData = []#SimpsDifference

    def Trapz(self):
        for i in range(self.NumberOfIterations):
            ListToAppend = []
            for j in range (self.NumberOfIterations):
                self.FunctionToIntegrate = FunctionHandling((i+1)*CoefficientIncrement, j).GetFunction()
                Integrate = Integration(self.FunctionToIntegrate, self.LowerBound, self.UpperBound, self.NumberOfSegments)
                Estimate = Integrate.Trapz()
                Difference = (Estimate - FunctionHandling((i+1)*CoefficientIncrement, j*PowerIncrement).GetValue(self.LowerBound, self.UpperBound))/(FunctionHandling((i+1)*CoefficientIncrement, j*PowerIncrement).GetValue(self.LowerBound, self.UpperBound))
                ListToAppend.append(Difference)
                #self.ZData.append(Difference)
            #ListToAppend = np.array(ListToAppend)
            ListToAppend = np.array(ListToAppend)
            self.ZData.append(ListToAppend)
    def Simps(self):
        for i in range(self.NumberOfIterations):
            ListToAppend = []
            for j in range (self.NumberOfIterations):
                self.FunctionToIntegrate = FunctionHandling((i+1)*CoefficientIncrement, j).GetFunction()
                Integrate = Integration(self.FunctionToIntegrate, self.LowerBound, self.UpperBound, self.NumberOfSegments)
                Estimate = Integrate.Simps()
                Difference = (Estimate - FunctionHandling((i+1)*CoefficientIncrement, j*PowerIncrement).GetValue(self.LowerBound, self.UpperBound))/(FunctionHandling((i+1)*CoefficientIncrement, j*PowerIncrement).GetValue(self.LowerBound, self.UpperBound))
                ListToAppend.append(Difference)
                #self.ZData.append(Difference)
            #ListToAppend = np.array(ListToAppend)
            ListToAppend = np.array(ListToAppend)
            self.ZData.append(ListToAppend)

TrapzData = Main(LowerBound, UpperBound, NumberOfSegments, NumberOfIterations)
TrapzData.Trapz()
X = np.array(getattr(TrapzData, 'XData'))
Y = np.array(getattr(TrapzData, 'YData'))
Z = np.array(getattr(TrapzData, 'ZData'))
TrapzGraph = Graph(X,Y,Z)
TrapzGraph = TrapzGraph.ThreeDimensionalGraph('Trapezium Method Error')

SimpsData = Main(LowerBound, UpperBound, NumberOfSegments, NumberOfIterations)
SimpsData.Simps()
X1 = np.array(getattr(SimpsData, 'XData'))
Y1 = np.array(getattr(SimpsData, 'YData'))
Z1 = np.array(getattr(SimpsData, 'ZData'))
SimpsGraph = Graph(X1,Y1,Z1)
SimpsGraph = SimpsGraph.ThreeDimensionalGraph('Simpson Rule Error')

Z2 = Z - Z1
DiffsGraph = Graph(X1,Y1,Z2)
DiffsGraph = DiffsGraph.ThreeDimensionalGraph('Trapezium Error - Simpson Error')

#print(Z)
print(Z2)

ZeroDivisionError: float division by zero